# Train Model
## Step 2 - Test your train.py with Docker

 USE SAGEMAKER.   Don't get confused - running jobs on the local SageMaker server isn't really what it was designed for.  It is designed to take your program and send it to outside resouces (using a Docker container)

In [ ]:
import os
import tensorflow as tf

In [ ]:
# pre-TensorFlow 2.0
# set up eager execution
tf.enable_eager_execution()
tf.set_random_seed(0)
tf.logging.set_verbosity(tf.logging.ERROR)

## Docker
Set up Docker - if this is an instance with a GPU, it will configure accordingly

In [ ]:
!/bin/bash ./sagemaker_docker_setup.sh

### create a local SageMaker estimator

code/train_model - this entire directory goes to the Docker image

In [ ]:
# an Estimator is a SageMaker class
# and, you're using the tensorflow flavor

import sagemaker
from sagemaker.tensorflow import TensorFlow

In [ ]:
model_dir = '/opt/ml/model'     # this is related to how it gets deployed in the Docker
                                # this is a SAGEMAKER thing - don't confuse with the model_dir 
                                # that we have inside our code
    
train_instance_type = 'local'   # local vs another server

hyperparameters = {'epochs': 5, 'batch_size': 128, 'learning_rate': 0.01}

# python modules that need to be loaded in the Docker
# what you have installed here has to be loaded there
env = { 'SAGEMAKER_REQUIREMENTS' : 'requirements.txt'}

# SageMaker Execution Role
# role = sagemaker.get_execution_role()
# since this doesn't work, get it manually from IAM
role = 'arn:aws:iam::586454201570:role/MLAM_SageMakerExecution'

In [ ]:
inputs = {'train' : f'file://code/tfrecords/train', 'test' : f'file://code/tfrecords/eval'}

In [ ]:
local_estimator = TensorFlow(entry_point='train.py',
                       source_dir='code',
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=role,
                       base_job_name='cfa_products_mobilenet_v1_SSD',
                       framework_version='1.13',
                       py_version='py3',
                       script_mode=True)

In [ ]:
role = sagemaker.get_execution_role()